In [ ]:
from os.path import basename
import scanpy as sc
import pandas as pd
import Spectra
import numpy as np
import json 
import scanpy as sc
from collections import OrderedDict
import scipy 
import pandas as pd
import matplotlib.pyplot as plt

#spectra imports 
import Spectra as spc
from Spectra import Spectra_util as spc_tl
from Spectra import K_est as kst
from Spectra import default_gene_sets

%run /home/zhongm/bao/shixiong/superscc/SuperSCC-version3.1.4.ipynb

/home/zhongm/miniconda3/envs/superscc/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# load data
file_path = "/mnt/disk1/zhongmin/superscc/胚胎数据/D023/未去批次效应数据/未去批次效应_D023乳腺数据2万.csv" # 改不同文件名以便读取不同文件
data = pd.read_csv(file_path, index_col=0)
data = data.select_dtypes("number")
data

metadata = pd.read_csv("/mnt/disk1/zhongmin/superscc/胚胎数据/D023/未去批次效应数据/D023乳腺数据_metadata2万.csv", index_col=0)
metadata = metadata.reindex(data.index)
metadata

In [ ]:
# do normalization
adata = sc.AnnData(data)
adata.obs = metadata
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

sc.pp.highly_variable_genes(adata, n_top_genes = 3000)

/home/zhongm/miniconda3/envs/superscc/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  


In [ ]:
# convert ID to symbols
id2symbol = pd.read_csv("/home/zhongm/daima/jupyter/human_id2symbol.csv")
print(id2symbol.columns)
adata = adata_id2symbol(adata,id2symbol)
adata.var_names_make_unique()

In [ ]:
# create a empty marker dict for ech cell type
adata_cell_types = adata.obs["broad_celltype"].unique().tolist()
annotations = {
    cell_type: {
        "marker": [] 
    } for cell_type in adata_cell_types
}
annotations["global"] = {
    "global": [] 
}

In [ ]:
# insert Spectra-collected markers 
annotations1 = Spectra.default_gene_sets.load()
annotations["global"] = annotations1["global"]  # 直接赋值替换

Cell type labels in gene set annotation dictionary and AnnData object are identical
Your gene set annotation dictionary is now correctly formatted.


In [ ]:
#run the Spectra model with default values
model = Spectra.est_spectra(
    adata=adata, 
    gene_set_dictionary=annotations, 
    use_highly_variable=True,
    cell_type_key="broad_celltype", 
    use_weights=True,
    lam=0.1, #varies depending on data and gene sets, try between 0.5 and 0.001
    delta=0.001, 
    kappa=None,
    rho=0.001, 
    use_cell_types=True,
    n_top_vals=50,
    label_factors=True, 
    overlap_threshold=0.2,
    clean_gs = True, 
    min_gs_num = 3,
    num_epochs=10000
)

Cell type labels in gene set annotation dictionary and AnnData object are identical
Your gene set annotation dictionary is now correctly formatted.


100%|██████████| 10000/10000 [6:43:14<00:00,  2.42s/it] 


In [19]:
adata.write_h5ad("/mnt/disk2/zhongm/superscc/spectra结果/D023/2025-4-28/task1/D023_细胞类型.h5ad")